In [1]:
# Добавляем директорию для равномерного импорта всего с корневой локации
import sys

sys.path.append('../')

# Импортируем библиотеку, которая автоматически настраивает коннектор для системы выполнения кода
from src.libs.LCTWrapTwin import start_module

In [2]:
# ЭТОТ БЛОК КОДА МОЖНО И НУЖНО МОДИФИЦИРОВАТЬ
# Как обычно, подсказки - в импортах

from typing import Literal
from src.libs.LCTWrapTwin.Modules.Handler import MissionHandler, TrustedHandler
import socket
import threading
import time
# # только для debug (фотки) ##
# from datetime import datetime
# import cv2
# import numpy as np
# import os
# # только для debug (фотки) ##

class UserMissionHandler(MissionHandler):

    @staticmethod
    def config_cyber_obstacles():
        # Сервисная функция, в которой вы ОБЯЗАНЫ передать конфигурацию киберпрепятствий.
        # Изменять можно только сами значения флагов - если функция вернёт что-то кроме данного объекта, возникнет ошибка.

        return {
            "CybP_01": False,  # постоянно (автопилот) - хэш автопилота
            "CybP_02": False,  # постоянно (неполадки связи, смешное сообщение)
            "CybP_03": False,  # постоянно (привода, поддел инфы о них) - хэш приводов
            "CybP_04": False,  # постоянно (ускорение щетки)
            "CybP_05": True,
            "CybP_06": False,  # постоянно (несанкц. ускорение)
        }
    
    def change_barrier_UDP(self, do: Literal["up"] | Literal["drop"]):  # Смена положения шлагбаума путем перехвата трафика [UDP] #

        def periodic_ap_barrier():
            future_result = 1 if do == "up" else 0
            while True:
                self.set_barrier_toggle()
                if UserTrustedHandler.check_CybP_05_UDP(self, future_result):
                    break

        ap_barrier = threading.Thread(target=periodic_ap_barrier, daemon=True)
        ap_barrier.start()

    def get_photo(self):  # Фото с камеры и сохранение в /Photos [debug] #
        # frame = self.get_camera_frame()
        
        # if frame is not None:
        #     save_dir = "Photos"

        #     # Генерируем уникальное имя файла с временной меткой
        #     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-3]  # миллисекунды
            
        #     # Сохраняем изображение
        #     image_filename = os.path.join(save_dir, f"frame_{timestamp}.jpg")
        #     cv2.imwrite(image_filename, frame)
        #     self.lg.log(f"Сохранение изображения")
            
        #     # Текстовый файл (только цвета пикселей)
        #     array_filename = os.path.join(save_dir, f"frame_{timestamp}.txt")
        #     np.savetxt(array_filename, frame.reshape(-1, frame.shape[-1]), fmt='%d', delimiter=',')
        #     self.lg.log(f"Сохранение массива")
        
        # return frame
        ...

    def intercept_UDP(self, name: str) -> int:  # Перехват трафика умных устройств [UDP] #

        try:
            ports = {
            "traffic_0": 5030,  # светофор 0 (самый ближний)
            "traffic_1": 5031,  # светофр 1 (средний)
            "traffic_2": 5032,  # светофор 2 (самый дальний)
            "barrier": 15000,   # шлагбаум
            "autobot": 10004    # автоматичский робот
            }

        except Exception as e:
            self.lg.log(f"Переданный в параметр name аргумент: {name}!")

        listen_port = ports[name]
        result = None
        data_autobot = set()

        def periodic_interceptor():
            nonlocal result
            
            interceptor_socket = None
            forward_socket = None
            
            try:
                interceptor_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                interceptor_socket.settimeout(0.3)
                interceptor_socket.bind(('127.0.0.1', listen_port))
                
                forward_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
                
                try:
                    data, client_address = interceptor_socket.recvfrom(1024)
                    decoded_data = data.decode('utf-8', errors='ignore').strip()
                    if name in tuple(ports.keys())[:-2]:
                        color = int(decoded_data[-2])
                        self.lg.log(f"{color}")
                        self.lg.log(f"{name} КРАСНЫЙ") if color == 1 else self.lg.log(f"{name} ЗЕЛЁНЫЙ")
                        if color == 2:
                            result = True
                    elif name in tuple(ports.keys())[-2]:
                        status = decoded_data[-2]
                        result = int(status)
                    elif name in tuple(ports.keys())[-1]:
                        result = decoded_data

                except socket.timeout:
                    self.lg.log("Отключение от порта, ДАННЫЕ НЕ ПОЛУЧЕНЫ!!!")
                    
                except Exception as e:
                    self.lg.log(f"Ошибка при получении данных: {e}")
                    
            except Exception as e:
                self.lg.log(f"Ошибка создания сокета: {e}")      
          
            finally:
                try:
                    if interceptor_socket:
                        interceptor_socket.close()
                    if forward_socket:
                        forward_socket.close()
                except:
                    pass
        
        interceptor_thread = threading.Thread(target=periodic_interceptor, daemon=True)
        interceptor_thread.start()
        interceptor_thread.join(timeout=0.25)
        return result

    def check_t_light_CV(self) -> int:  # Проверка светофора на зеленый/красный [КЗ] #

        velocity = 20
        
        while True:
            frame = self.get_camera_frame()
            center_row = frame[240, :]
            
            green_pixels = 0
            red_pixels = 0
            
            for x in range(len(center_row)):
                pixel = center_row[x]
                b, g, r = pixel[0].astype(int), pixel[1].astype(int), pixel[2].astype(int)
                
                is_green = (b in range(40-velocity, 40+velocity)) and (g in range(200-velocity, 200+velocity)) and (r in range(40-velocity, 40+velocity))
                
                is_red = (b in range(40-velocity, 40+velocity)) and (g in range(40-velocity, 40+velocity)) and (r in range(200-velocity, 200+velocity))
                
                if is_green:
                    green_pixels += 1      
                elif is_red:
                    red_pixels += 1
                
            if green_pixels > red_pixels:
                self.lg.log("Световор ЗЕЛЕНЫЙ")
                break
            else:
                self.lg.log("Световор КРАСНЫЙ")
                time.sleep(1.0)

        return True 

    def check_autobot_UDP(self):  # Проверка положения автобота перехватом трафика [UDP] #
        while True:  # если робот стоит на месте, если двигается -> запускаем другой цикл и ждем остановки
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                self.lg.log("Робот скоро поедет. Ожидаем...")
                continue
            else:
                break
        while True:
            new_1 = self.intercept_UDP("autobot")
            time.sleep(1.0)
            new_2 = self.intercept_UDP("autobot")
            if new_1 == new_2:
                self.lg.log("Робот остановился, можно ехать")
                break
        self.lg.log(f"{new_2}")
        return True

    def check_pedestrian_CV(self):  # Проверка положения пешехода [КЗ] #
        BGR_TARGET_1 = (65, 110, 200)  # приблизительные bgr для конуса без тени
        BGR_TARGET_2 = (80, 80, 80)  # приблизительные bgr для конуса в тени
        x, y = 323, 426
        velocity = 55
        
        while True:
            frame = self.get_camera_frame()
            pixels = frame[y, x-2:x+3]
            
            mas_1 = [[pix[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)] for pix in pixels]
            mas_2 = [[pix[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)] for pix in pixels]
            
            result_1 = any([all(i) for i in mas_1])
            result_2 = any([all(i) for i in mas_2])
            
            if result_1 or result_2:
                self.lg.log(f"Обнаружен пешеход. Можно продолжать движение")
                break
            else:
                time.sleep(0.2)
                continue
        return True

    def change_barrier_CV(self):  # Смена положения шлагбаума (только подъем; закрытие - надо разворачитваться к шлагбауму)[КЗ] #
        
        BGR_TARGET_1 = [44, 43, 117]  # без тени
        BGR_TARGET_2 = [50, 50, 50]  # с учетом тени
        x, y = 301, 273
        velocity = 30
        
        def check_status():
            frame = self.get_camera_frame()
            pixels = frame[y, x-20:x+20]
            for pixel in pixels:
                mass_1 = [pixel[i] in range(BGR_TARGET_1[i] - velocity, BGR_TARGET_1[i] + velocity) for i in range(3)]
                mass_2 = [pixel[i] in range(BGR_TARGET_2[i] - velocity, BGR_TARGET_2[i] + velocity) for i in range(3)]
                if all(mass_1) or all(mass_2):
                    self.lg.log(f"{pixel}")
                    self.lg.log(f"Шлагбаум ЗАКРЫТ")
                    return 0
            self.lg.log(f"Шлагбаум ОТКРЫТ")
            return 1
        
        start_status = check_status()
        
        while True:
            self.set_barrier_toggle()
            time.sleep(2.0)
            potential_status = check_status()
            if UserTrustedHandler.check_CybP_05_CV(self, start_status, potential_status):
                break



    def mission_code(self):
        
        ### --- МАРШРУТЫ ДВИЖЕНИЯ --- ###
        
        def mine():
            self.set_robot_speed(0.1)
            self.set_brush_speed(100)

            self.ap_hook.do_rotate({"x": 0.6, "y": 0.6})
            self.ap_hook.do_move({"x": 0.6, "y": 0.6})

            self.ap_hook.do_rotate({"x": 0.8, "y": 0.6})
            self.ap_hook.do_move({"x": 1.4, "y": 0.6})

            self.ap_hook.do_rotate({"x": 1.4, "y": 1.0})
            self.ap_hook.do_move({"x": 1.4, "y": 1.0})
            self.ap_hook.do_rotate({"x": 1.4, "y": 0.2})
            
            # Перед светофором 0
            self.check_t_light_CV()

            self.ap_hook.do_rotate({"x": 1.4, "y": 1.2})
            self.ap_hook.do_move({"x": 1.4, "y": 2.2})
            self.ap_hook.do_rotate({"x": 1.8, "y": 2.2})
            self.ap_hook.do_move({"x": 1.8, "y": 2.2})
            self.ap_hook.do_rotate({"x": 1.4, "y": 2.2})
            
            # Перед светофором 1
            self.check_t_light_CV()


        def barrier():
            self.set_robot_speed(0.1)
            self.set_brush_speed(100)

            self.ap_hook.do_rotate({"x": 0.6, "y": 1.2})
            self.ap_hook.do_move({"x": 0.6, "y": 1.2})

            self.ap_hook.do_rotate({"x": 0.6, "y": 1.4})
            self.ap_hook.do_move({"x": 0.6, "y": 1.4})  # перед шлаг
            self.change_barrier_CV()  # закрыли
            self.change_barrier_CV()  # октрыли
            self.ap_hook.do_move({"x": 0.6, "y": 2.2})
            self.change_barrier_UDP("drop")  # закрыли

            self.ap_hook.do_move({"x": 0.6, "y": 2.35})  # перед пешеходником
            self.check_pedestrian_CV()
            self.ap_hook.do_move({"x": 0.6, "y": 3.0})

            self.ap_hook.do_rotate({"x": 0.6, "y": 2.85})

            self.ap_hook.do_move({"x": 0.6, "y": 2.85})  # перед пешеходником
            self.check_pedestrian_CV()
            self.ap_hook.do_move({"x": 0.6, "y": 2.2})  # перед шлаг
            self.change_barrier_CV()  # октрыли
            self.ap_hook.do_move({"x": 0.6, "y": 1.4})
            self.change_barrier_UDP("drop")  # закрыли


        def brovkin():
            self.lg.log("Сообщение обычное")
            self.lg.warn("Сообщение о предупреждении")
            self.lg.error("Сообщение об ошибке")

            self.lg.log("Тестовое сообщение блока кода для выполнения заезда")
            self.set_robot_speed(0.24)
            self.set_brush_speed(100)
            self.change_barrier_UDP("drop")  # Сразу закрываем шлагбаум дистанционно с помощью пакетов
            self.ap_hook.do_rotate({"x": 0.6, "y": 1})
            self.ap_hook.do_move({"x": 0.6, "y": 1})
            self.ap_hook.do_rotate({"x": 0.68, "y": 1.4})
            self.ap_hook.do_move({"x": 0.68, "y": 1.4})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.68, "y": 2.35})  # Пешеход верхний
            self.check_pedestrian_CV()
            self.change_barrier_UDP("drop")  # Снова закрываем шлагбаум дистанционно с помощью пакетов

            self.ap_hook.do_move({"x": 0.68, "y": 2.92})
            self.ap_hook.do_rotate({"x": 1, "y":2.92})
            self.ap_hook.do_move({"x": 1, "y": 2.92})  # Возможное столкновение с роботом 
            self.check_autobot_UDP()
            self.ap_hook.do_move({"x": 2.28, "y":2.92})
            self.ap_hook.do_rotate({"x": 2.28, "y":3.32})
            self.ap_hook.do_move({"x": 2.28, "y":3.32})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.32})
            self.ap_hook.do_move({"x": 2.6, "y":3.32})
            self.ap_hook.do_rotate({"x": 3.0, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.48})
            self.ap_hook.do_move({"x": 2.6, "y":3.48})
            self.ap_hook.do_rotate({"x": 2.6, "y":3.48})
            self.ap_hook.do_move({"x": 2.2, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.4})
            self.ap_hook.do_move({"x": 2.12, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.12, "y":3.08})
            self.ap_hook.do_move({"x": 2.12, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.8, "y":3.08})
            self.ap_hook.do_move({"x": 1.32, "y":3.08})
            self.ap_hook.do_rotate({"x": 1.32, "y":2.2})
            self.ap_hook.do_move({"x": 1.32, "y":2.12})
            self.ap_hook.do_rotate({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 1.8, "y":2.12})
            self.ap_hook.do_move({"x": 2.2, "y":2.12})
            self.ap_hook.do_rotate({"x": 2.6, "y":2.2})
            self.ap_hook.do_move({"x": 2.6, "y":2.28})
            self.ap_hook.do_move({"x": 2.2, "y":2.28})
            self.ap_hook.do_move({"x": 2.2, "y":2.28})
            self.ap_hook.do_move({"x": 1.8, "y":2.28})
            self.ap_hook.do_rotate({"x": 1.32, "y":2.2})
            self.ap_hook.do_move({"x": 1.32, "y":2.2})
            self.ap_hook.do_move({"x": 1.32, "y":1})
            self.ap_hook.do_rotate({"x": 1.4, "y":0.6})
            self.ap_hook.do_move({"x": 1.4, "y":0.6})
            self.ap_hook.do_rotate({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1})
            self.ap_hook.do_move({"x": 1.48, "y":1.4})
            self.ap_hook.do_move({"x": 1.48, "y":1.8})
            self.ap_hook.do_move({"x": 1.48, "y":2.2})
            self.ap_hook.do_move({"x": 1.48, "y":2.6})
            self.ap_hook.do_rotate({"x": 1.4, "y":3.08})
            self.ap_hook.do_move({"x": 1.4, "y":3.08})
            self.ap_hook.do_move({"x": 1.0, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":3.08})
            self.ap_hook.do_move({"x": 0.52, "y":3.08})
            self.ap_hook.do_rotate({"x": 0.52, "y":2.85})
            self.ap_hook.do_move({"x": 0.52, "y":2.85})  # Верхний пешехов        
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 0.52, "y":2.2})  # Шлагбаум 
            self.change_barrier_CV()

            self.ap_hook.do_move({"x": 0.52, "y":1.8})
            self.ap_hook.do_move({"x": 0.52, "y":1.4})
            self.ap_hook.do_move({"x": 0.52, "y":1})
            self.ap_hook.do_rotate({"x": 0.6, "y":0.6})
            self.ap_hook.do_move({"x": 0.6, "y":0.6})
            self.ap_hook.do_rotate({"x": 1, "y":0.52})
            self.ap_hook.do_move({"x": 1, "y":0.52})

            self.ap_hook.do_move({"x": 1.4, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52})
            self.ap_hook.do_move({"x": 1.8, "y":0.52}) 
            self.ap_hook.do_move({"x": 1.9, "y":0.52})
            self.ap_hook.do_move({"x": 1.95, "y":0.52})  # Пешеход нижний 
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 2.2, "y":0.52})
            self.ap_hook.do_move({"x": 2.6, "y":0.52})    
            self.ap_hook.do_rotate({"x": 3.08, "y":0.52})
            self.ap_hook.do_move({"x": 3.08, "y":0.52})
            self.ap_hook.do_rotate({"x": 3.08, "y":1})  # Возможное столкновение с роботом
            self.check_autobot_UDP()

            self.ap_hook.do_move({"x": 3.08, "y":1})
            self.ap_hook.do_move({"x": 3.08, "y":1.4})
            self.ap_hook.do_move({"x": 3.08, "y":1.8})

            self.ap_hook.do_move({"x": 3.08, "y":2.2})
            self.ap_hook.do_move({"x": 3.08, "y":2.6})
            self.ap_hook.do_move({"x": 3.08, "y":3.0})  # !!! Возможное столкновение с роботом !!!
            # self.check_autobot_UDP() !!! НЕ ЗДЕСЬ !!!

        
            self.ap_hook.do_rotate({"x": 3, "y":3.4})
            self.ap_hook.do_move({"x": 3.0, "y":3.4})
            self.ap_hook.do_rotate({"x": 2.92, "y":3})
            self.ap_hook.do_move({"x": 2.92, "y":3})
            self.ap_hook.do_rotate({"x": 2.92, "y":3.0})
            self.ap_hook.do_move({"x": 2.92, "y":3.0})
            self.ap_hook.do_move({"x": 2.92, "y":1.0})
            self.ap_hook.do_move({"x": 2.92, "y":0.6})
            self.ap_hook.do_rotate({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.6, "y":0.68})
            self.ap_hook.do_move({"x": 2.45, "y":0.68})  # Пешеход нижний
            self.check_pedestrian_CV()

            self.ap_hook.do_move({"x": 1.8, "y":0.68})
            self.ap_hook.do_move({"x": 1.4, "y":0.68}) 
            self.ap_hook.do_move({"x": 1.0, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.68, "y":0.68})
            self.ap_hook.do_move({"x": 0.68, "y":0.68})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})
            self.ap_hook.do_move({"x": 0.6, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.2, "y":1.0})
            self.ap_hook.do_move({"x": 0.2, "y":1.0})
            self.ap_hook.do_rotate({"x": 0.6, "y":1.0})      

        ### --- МАРШРУТЫ ДВИЖЕНИЯ --- ###

        
        # ЗАПУСК МАРШРУТА #
        # mine()
        # barrier()
        brovkin()



class UserTrustedHandler(TrustedHandler):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
    
    def trusted_code(self):
        pass

    def check_CybP_05_CV(ap: any, start_status: 0 | 1, potential_status: 0 | 1):  # Проверка активности CybP_05 [КЗ] #
        if potential_status != start_status or potential_status == 1:
            ap.lg.log("Шлагбаум ОТКРЫТ")
            return True
        else:
            ap.lg.log("Положение шлагбаума не изменено, активна CybP_05")
            return False

    def check_CybP_05_UDP(ap: any, future_result: int):  # Проверка активности CybP_05 [UDP] #
        time.sleep(0.3)
        if future_result == ap.intercept_UDP("barrier"):
            ap.lg.log("Положение шлагбаума ИЗМЕНЕНО")
            return True
        else:
            ap.lg.log("Активна CybP_05")
            return False
       
    def make_next_short_message(self, prev_message: str):
        
        """Генерация коротких сообщений для АСО"""
        # Ваш код для создания коротких сообщений
        some_message = "exactly_correct_short_message"
        return some_message


In [3]:
start_module(UserMissionHandler, UserTrustedHandler)

[2025-09-30 16:00:57]::[AwC]::INIT::LCT-WRAP-CLIENT, версия 1.2.0
[2025-09-30 16:00:57]::[AwC]::INIT::UDP(c) <- (uid=aLPC) инициализирован.
[2025-09-30 16:00:57]::[AwC]::INIT::UDP(c) <- (uid=hhTZ) инициализирован.
[2025-09-30 16:00:57]::[AwC]::INIT::Загрузка завершена.
[2025-09-30 16:00:57]::[AwC]::INIT::UDP(c) -> (uid=DKaX) инициализирован.
[2025-09-30 16:00:57]::[AwC]::INIT::Адрес сервера (command_receiver): http://127.0.0.1:13500
[2025-09-30 16:00:57]::[AwC]::INFO::(AP) Заезд инициализирован - ожидание старта
[2025-09-30 16:00:57]::[AwC]::INFO::(TM) Заезд инициализирован - ожидание старта
[2025-09-30 16:01:01]::[AwC]::INFO::Код заезда инициализирован
[2025-09-30 16:01:01]::[AwC]::INFO::Сообщение обычное
[2025-09-30 16:01:02]::[AwC]::WARN::Сообщение о предупреждении
[2025-09-30 16:01:02]::[AwC]::ERROR::Сообщение об ошибке
[2025-09-30 16:01:02]::[AwC]::INFO::Доверенный код инициализирован
[2025-09-30 16:01:02]::[AwC]::INFO::Тестовое сообщение блока кода для выполнения заезда
[2025-09-